BBPSSW CON UN GIRO DI DISTILLAZIONE

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit_aer.noise import NoiseModel, depolarizing_error

rounds = 10000
sim = Aer.get_backend("aer_simulator")

depol_values = [0.10, 0.15, 0.20, 0.25, 0.30]

for p in depol_values:
    qc = QuantumCircuit(4, 4)

    qc.h(0)  # A_0
    qc.cx(0, 1)  # B_0
    qc.h(2)  # A_1
    qc.cx(2, 3)  # B_1

    qc.cx(0, 2)
    qc.cx(1, 3)

    qc.measure(2, 2)
    qc.measure(3, 3)
    qc.measure(0, 0)
    qc.measure(1, 1)

 
    noise = NoiseModel()
    noise.add_all_qubit_quantum_error(depolarizing_error(p, 2), ['cx'])
    noise.add_all_qubit_quantum_error(depolarizing_error(p, 1), ['h', 'x'])

    
    job = sim.run(transpile(qc, sim, basis_gates=noise.basis_gates, optimization_level=0),shots=rounds,noise_model=noise).result()

    counts = job.get_counts()

    correct = 0
    passed_distillation = 0

    for bitstring, count in counts.items(): 
        b0 = bitstring[3]
        b1 = bitstring[2]
        b2 = bitstring[1]
        b3 = bitstring[0]

        if b0 == b1:
            passed_distillation += count
            if b2 == b3:
                correct += count

    fidelity = passed_distillation / rounds

    
    fidelity_distilled = correct / passed_distillation

    F = fidelity
    nu = F**2 + (1/9) * (1 - F)**2
    de = F**2 + (2/3) * F * (1 - F) + (5/9) * (1 - F)**2
    expected_fidelity_distilled = nu / de

    print("=====================================================")
    print(f"Depolarizing error p = {p}")
    print(f"Fidelity entanglement iniziale: {fidelity}")
    print(f"Fidelity after distillation (sim): {fidelity_distilled}")
    print(f"Fidelity after distillation (expected): {expected_fidelity_distilled}")


Depolarizing error p = 0.1
Fidelity entanglement: 0.8666
Fidelity after distillation (sim): 0.8964920378490653
Fidelity after distillation (expected): 0.8985875036899506
Depolarizing error p = 0.15
Fidelity entanglement: 0.8078
Fidelity after distillation (sim): 0.8531814805644962
Fidelity after distillation (expected): 0.8455717912637875
Depolarizing error p = 0.2
Fidelity entanglement: 0.7648
Fidelity after distillation (sim): 0.8066161087866108
Fidelity after distillation (expected): 0.8035450157134754
Depolarizing error p = 0.25
Fidelity entanglement: 0.7083
Fidelity after distillation (sim): 0.7587180573203445
Fidelity after distillation (expected): 0.7443458445179376
Depolarizing error p = 0.3
Fidelity entanglement: 0.6723
Fidelity after distillation (sim): 0.7069760523575784
Fidelity after distillation (expected): 0.7044854781810457


BBPSSW CON DUE GIRI DI DISTILLAZIONE, quiskit non permetter di fare circuiti modificabili quindi ogni aggiunta di qubit devo fare un altro circuito

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit_aer.noise import NoiseModel, depolarizing_error

rounds = 10000
sim = Aer.get_backend("aer_simulator")

depol_values = [0.05, 0.08, 0.11, 0.14, 0.17]

for p in depol_values:
    qc = QuantumCircuit(6, 6)

    qc.h(0)  
    qc.cx(0, 1)  
    qc.h(2)  
    qc.cx(2, 3) 
    qc.h(4)  
    qc.cx(4, 5)  

    qc.cx(0, 2)
    qc.cx(1, 3)
    qc.cx(0, 4)
    qc.cx(1, 5)

    qc.measure(2, 2)
    qc.measure(3, 3)
    qc.measure(4, 4)
    qc.measure(5, 5)
    qc.measure(0, 0)
    qc.measure(1, 1)

 
    noise = NoiseModel()
    noise.add_all_qubit_quantum_error(depolarizing_error(p, 2), ['cx'])
    noise.add_all_qubit_quantum_error(depolarizing_error(p, 1), ['h', 'x'])

    
    job = sim.run(transpile(qc, sim, basis_gates=noise.basis_gates, optimization_level=0),shots=rounds,noise_model=noise).result()

    counts = job.get_counts()

    correct = 0
    passed_distillation = 0
    first_passed  = 0 # questo serve a stimare la fidelity
    for bitstring, count in counts.items(): 
        b0 = bitstring[5]
        b1 = bitstring[4]
        b2 = bitstring[3]
        b3 = bitstring[2]
        b4 = bitstring[1]
        b5 = bitstring[0]

        if b0 == b1 and b2 == b3:
            passed_distillation += count
            if b4 == b5:
                correct += count

        if b0 == b1:
            first_passed += count


    fidelity = first_passed / rounds
    fidelity_distilled = correct / passed_distillation

    F = fidelity
    nu = F**2 + (1/9) * (1 - F)**2
    de = F**2 + (2/3) * F * (1 - F) + (5/9) * (1 - F)**2

    F1 = nu / de
    nu = F1**2 + (1/9) * (1 - F1)**2
    de = F1**2 + (2/3) * F1 * (1 - F1) + (5/9) * (1 - F1)**2
    expected_fidelity_distilled = nu/de
    print("=====================================================")
    print(f"Depolarizing error p = {p}")
    print(f"Fidelity entanglement iniziale: {fidelity}")
    print(f"Fidelity after distillation (sim): {fidelity_distilled}")
    print(f"Fidelity after distillation (expected): {expected_fidelity_distilled}")


ORA CON 4 GIRI, aggiungo più rumore su altre coppie

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit_aer.noise import NoiseModel, depolarizing_error
import numpy as np
rounds = 10000
sim = Aer.get_backend("aer_simulator")

depol_values = [0.05, 0.08, 0.11, 0.14, 0.17]

for p in depol_values:
    qc = QuantumCircuit(10, 10)

    qc.h(0)  
    qc.cx(0, 1)  
    qc.h(2)  
    qc.cx(2, 3) 
    qc.h(4)  
    qc.cx(4, 5)  
    qc.h(6)  
    qc.cx(6, 7) 
    qc.h(8)  
    qc.cx(8, 9)  

    qc.cx(0, 2)
    qc.cx(1, 3)
    qc.cx(0, 4)
    qc.cx(1, 5)
    qc.cx(0, 6)
    qc.cx(1, 7)
    qc.cx(0, 8)
    qc.cx(1, 9)

    #rumore aggiunto per dargli circa la stessa fidelity nei casi precedenti era di meno la differenza
    #perchè applico il depolarizing error di più 0 e 1 che altri qubit di base
    #depol_error * n_giri è la differza che c'è 
    qc.x(2)
    qc.x(3)
    qc.x(4)
    qc.x(5)
    qc.x(6)
    qc.x(7)
    qc.x(8)
    qc.x(9)
    qc.x(2)
    qc.x(3)
    qc.x(4)
    qc.x(5)
    qc.x(6)
    qc.x(7)
    qc.x(8)
    qc.x(9)
    #########################################################################################à#
    qc.measure(2, 2)
    qc.measure(3, 3)
    qc.measure(4, 4)
    qc.measure(5, 5)
    qc.measure(6, 6)
    qc.measure(7, 7)
    qc.measure(8, 8)
    qc.measure(9, 9)  
    qc.measure(0, 0)
    qc.measure(1, 1)

 
    noise = NoiseModel()
    noise.add_all_qubit_quantum_error(depolarizing_error(p, 2), ['cx'])
    noise.add_all_qubit_quantum_error(depolarizing_error(p, 1), ['h', 'x'])

    
    job = sim.run(transpile(qc, sim, basis_gates=noise.basis_gates, optimization_level=0),shots=rounds,noise_model=noise).result()

    counts = job.get_counts()

    correct = 0
    passed_distillation = 0
    first_passed  = 0 # questo serve a stimare la fidelity
    for bitstring, count in counts.items(): 
        b0 = bitstring[9]
        b1 = bitstring[8]
        b2 = bitstring[7]
        b3 = bitstring[6]
        b4 = bitstring[5]
        b5 = bitstring[4]
        b6 = bitstring[3]
        b7 = bitstring[2]
        b8 = bitstring[1]
        b9 = bitstring[0]

        if b0 == b1 and b2 == b3 and b4 == b5 and b6 == b7:
            passed_distillation += count
            if b8 == b9:
                correct += count

        if b0 == b1:
            first_passed += count


    fidelity = first_passed / rounds
    fidelity_distilled = correct / passed_distillation

    F = fidelity
    for i in range(4):
        nu = F**2 + (1/9) * (1 - F)**2
        de = F**2 + (2/3) * F * (1 - F) + (5/9) * (1 - F)**2
        F = nu /de
    
    # Update F for the
    expected_fidelity_distilled = F
    print("=====================================================")
    print(f"Depolarizing error p = {p}")
    print(f"Fidelity entanglement iniziale: {fidelity}")
    print(f"Fidelity after distillation (sim): {fidelity_distilled}")
    print(f"Fidelity after distillation (expected): {expected_fidelity_distilled}")


/home/ganger/Scrivania/ALL_project-main./.venv/lib/python3.12/site-packages/qiskit/compiler/transpiler.py:269: UserWarning: Providing `coupling_map` and/or `basis_gates` along with `backend` is not recommended, as this will invalidate the backend's gate durations and error rates.
  pm = generate_preset_pass_manager(


Depolarizing error p = 0.05
Fidelity entanglement: 0.8203
Fidelity after distillation (sim): 0.8628043026986224
Fidelity after distillation (expected): 0.9412273521488385
Depolarizing error p = 0.08
Fidelity entanglement: 0.7333
Fidelity after distillation (sim): 0.8096668487165484
Fidelity after distillation (expected): 0.8819382944244316
Depolarizing error p = 0.11
Fidelity entanglement: 0.6771
Fidelity after distillation (sim): 0.7436384352449678
Fidelity after distillation (expected): 0.8230389976167853
Depolarizing error p = 0.14
Fidelity entanglement: 0.6381
Fidelity after distillation (sim): 0.7120470127326151
Fidelity after distillation (expected): 0.7691980769491171
Depolarizing error p = 0.17
Fidelity entanglement: 0.5963
Fidelity after distillation (sim): 0.6523136246786633
Fidelity after distillation (expected): 0.6983304749846236


Ora che lo abbiamo messo sotto pressione proviamo se si comporta meglio DJEMPS

In [36]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit_aer.noise import NoiseModel, depolarizing_error
import numpy as np
rounds = 10000
sim = Aer.get_backend("aer_simulator")

depol_values = [0.05, 0.08, 0.11, 0.14, 0.17]

for p in depol_values:
    qc = QuantumCircuit(10, 10)

    qc.h(0)  
    qc.cx(0, 1)  
    qc.h(2)  
    qc.cx(2, 3) 
    qc.h(4)  
    qc.cx(4, 5)  
    qc.h(6)  
    qc.cx(6, 7) 
    qc.h(8)  
    qc.cx(8, 9)  

    qc.cx(0, 2)
    qc.cx(1, 3)
    qc.cx(0, 4)
    qc.cx(1, 5)
    qc.cx(0, 6)
    qc.cx(1, 7)
    qc.cx(0, 8)
    qc.cx(1, 9)

    #rumore aggiunto per dargli circa la stessa fidelity nei casi precedenti era di meno la differenza
    #perchè applico il depolarizing error di più 0 e 1 che altri qubit di base
    #depol_error * n_giri è la differza che c'è 
    qc.x(2)
    qc.x(3)
    qc.x(4)
    qc.x(5)
    qc.x(6)
    qc.x(7)
    qc.x(8)
    qc.x(9)
    qc.x(2)
    qc.x(3)
    qc.x(4)
    qc.x(5)
    qc.x(6)
    qc.x(7)
    qc.x(8)
    qc.x(9)
    ##########################################################################################
    #ora applichiamo rotazioni
    qc.rx(np.pi/2, 0)
    qc.rx(-np.pi/2, 1)
    qc.rx(np.pi/2, 2)
    qc.rx(-np.pi/2, 3)
    qc.rx(np.pi/2, 4)
    qc.rx(-np.pi/2, 5)
    qc.rx(np.pi/2, 6)
    qc.rx(-np.pi/2, 7)
    qc.rx(np.pi/2, 8)
    qc.rx(-np.pi/2, 9)
    #########################################################################################
    qc.measure(2, 2)
    qc.measure(3, 3)
    qc.measure(4, 4)
    qc.measure(5, 5)
    qc.measure(6, 6)
    qc.measure(7, 7)
    qc.measure(8, 8)
    qc.measure(9, 9)  
    qc.measure(0, 0)
    qc.measure(1, 1)

 
    noise = NoiseModel()
    noise.add_all_qubit_quantum_error(depolarizing_error(p, 2), ['cx'])
    noise.add_all_qubit_quantum_error(depolarizing_error(p, 1), ['h', 'x'])

    
    job = sim.run(transpile(qc, sim, basis_gates=noise.basis_gates, optimization_level=0),shots=rounds,noise_model=noise).result()

    counts = job.get_counts()

    correct = 0
    passed_distillation = 0
    first_passed  = 0 # questo serve a stimare la fidelity
    for bitstring, count in counts.items(): 
        b0 = bitstring[9]
        b1 = bitstring[8]
        b2 = bitstring[7]
        b3 = bitstring[6]
        b4 = bitstring[5]
        b5 = bitstring[4]
        b6 = bitstring[3]
        b7 = bitstring[2]
        b8 = bitstring[1]
        b9 = bitstring[0]

        if b0 == b1 and b2 == b3 and b4 == b5 and b6 == b7:
            passed_distillation += count
            if b8 == b9:
                correct += count

        if b0 == b1:
            first_passed += count


    fidelity = first_passed / rounds
    fidelity_distilled = correct / passed_distillation

    F = fidelity

   
    for i in range(4):
        nu = F**2 + (1/9) * (1 - F)**2
        de = F**2 + (2/3) * F * (1 - F) + (5/9) * (1 - F)**2
        F = nu /de
    expected_fidelity_distilled = F


    print("=====================================================")
    print(f"Depolarizing error p = {p}")
    print(f"Fidelity entanglement iniziale: {fidelity}")
    print(f"Fidelity after distillation (sim): {fidelity_distilled}")
    print(f"Fidelity after distillation when not applying rotation: {expected_fidelity_distilled}")


Depolarizing error p = 0.05
Fidelity entanglement iniziale: 0.7069
Fidelity after distillation (sim): 0.8672752808988764
Fidelity after distillation when not applying rotation: 0.8567511173910967
Depolarizing error p = 0.08
Fidelity entanglement iniziale: 0.6177
Fidelity after distillation (sim): 0.7791208791208791
Fidelity after distillation when not applying rotation: 0.7363043074265313
Depolarizing error p = 0.11
Fidelity entanglement iniziale: 0.5661
Fidelity after distillation (sim): 0.6938885884261763
Fidelity after distillation when not applying rotation: 0.639382555406838
Depolarizing error p = 0.14
Fidelity entanglement iniziale: 0.5373
Fidelity after distillation (sim): 0.6201959306706858
Fidelity after distillation when not applying rotation: 0.5790892180261568
Depolarizing error p = 0.17
Fidelity entanglement iniziale: 0.5225
Fidelity after distillation (sim): 0.5865921787709497
Fidelity after distillation when not applying rotation: 0.5474864350559212
